In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp


### Define the model, with trainable variables

In [19]:
def linear_mixed_effects_model(features):
  # Set up fixed effects and other parameters.
  # Note: these are trainable!
  intercept = tf.get_variable("intercept", [])  # alpha in eq
  effect_service = tf.get_variable("effect_service", [])  # beta in eq
  stddev_students = tf.exp(
      tf.get_variable("stddev_unconstrained_students", []))  # sigma in eq
  stddev_instructors = tf.exp(
      tf.get_variable("stddev_unconstrained_instructors", [])) # sigma in eq
  stddev_departments = tf.exp(
      tf.get_variable("stddev_unconstrained_departments", [])) # sigma in eq

  # Set up random effects.
  effect_students = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_students),
      scale_identity_multiplier=stddev_students,
      name="effect_students")
  effect_instructors = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_instructors),
      scale_identity_multiplier=stddev_instructors,
      name="effect_instructors")
  effect_departments = ed.MultivariateNormalDiag(
      loc=tf.zeros(num_departments),
      scale_identity_multiplier=stddev_departments,
      name="effect_departments")

  # Set up likelihood given fixed and random effects.
  # Note we use `tf.gather` instead of matrix-multiplying a design matrix of
  # one-hot vectors. The latter is memory-intensive if there are many groups.
  ratings = ed.Normal(
      loc=(effect_service * features["service"] +
           tf.gather(effect_students, features["students"]) +
           tf.gather(effect_instructors, features["instructors"]) +
           tf.gather(effect_departments, features["departments"]) +
           intercept),
      scale=1.,
      name="ratings")
  return ratings

model_template = tf.make_template("model", linear_mixed_effects_model)

NameError: name 'linear_mixed_effects_model' is not defined

### Define training operations
* a "loss minimization" step
* but since this is bayesian, also 

In [ ]:
# define operations for sampling random effect values
# samples are chosen that optimize the joint probability of y, X, and the effect values

next_state, kernel_results = hmc.one_step(
      current_state=current_state,
      previous_kernel_results=hmc.bootstrap_results(current_state))


current_state = ?

next_state, kernel_results = hmc.one_step(
    current_state=current_state,
    
)

# kernel_results... used for what?
# next_state is used for updating effect values, when sampled

expectation_update = tf.group(  # tf.group - Create an op that groups multiple operations
    effect_students.assign(next_state[0]),  # 
    effect_instructors.assign(next_state[1]),
    effect_departments.assign(next_state[2]),
)


In [18]:
'''
define bayesian fitting steps:
  * minimization_update: update trainable variables
    * it minimizes the joint log likelihood function of X, y, and parameter values
    * this depends on the distributions learned for the parameter values, which vary by effect means
  * expectation_update: update group effects
    * this is a monte carlo sampling operation
'''

# define the minimization step
# this improves fits of fixed effects, given the random effects
# this with block makes sure that random effects are updated before any of these operations
with tf.control_dependencies([expectation_update]):
    # tf.control_dependencies - adds dependencies to ops you create in the with block
    # `loss` = negative log of probability, given group effects
    loss = -target_log_prob_fn(
        effect_students,
        effect_instructors,
        effect_departments)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    # minimize according to effects passed to loss
    minimization_update = optimizer.minimize(loss)

'''
operations defined here...
expectation_update -> loss -> minimization_update

'''

NameError: name 'effect_students' is not defined

In [ ]:
expectation_update = tf.group

In [10]:
num_students = max(features_train['students']) + 1
num_instructors = max(features_train['instructors']) + 1
num_departments = max(features_train['departments']) + 1
num_observations = train.shape[0]

In [11]:
init = tf.global_variables_initializer()

num_warmup_iters = 1000
num_iters = 1500
num_accepted = 0
effect_students_samples = np.zeros([num_iters, num_students])
effect_instructors_samples = np.zeros([num_iters, num_instructors])
effect_departments_samples = np.zeros([num_iters, num_departments])
loss_history = np.zeros([num_iters])

sess = tf.Session()
sess.run(init)

In [17]:
# Run training.
for t in range(num_iters):
  for _ in range(5):  # run 5 MCMC iterations before every joint EM update
    _ = sess.run(expectation_update)
  [
      _,
      _,
      effect_students_val,
      effect_instructors_val,
      effect_departments_val,
      is_accepted_val,
      loss_val,
  ] = sess.run([
      expectation_update,
      minimization_update,
      effect_students,
      effect_instructors,
      effect_departments,
      kernel_results.is_accepted,
      loss,
  ])
  effect_students_samples[t, :] = effect_students_val
  effect_instructors_samples[t, :] = effect_instructors_val
  effect_departments_samples[t, :] = effect_departments_val
  num_accepted += is_accepted_val
  loss_history[t] = loss_val
  if t % 500 == 0 or t == num_iters - 1:
    print("Iteration: {:>4} Acceptance Rate: {:.3f} Loss: {:.3f}".format(
        t, num_accepted / (t + 1), loss_val))

TypeError: Fetch argument None has invalid type <class 'NoneType'>